# Introduction

This notebook shows how to use ZKLoRA to prove the forward pass of a LoRA model.

# Setup

In order to use this notebook, first install the zklora package:

```bash
pip install zklora
```

Or, if you are running this notebook locally and from the `examples` directory in this repository, you have to let the notebook know where to find the zklora package.

In [11]:
import sys
sys.path.append('..')

Now we can import the necessary functions from the zklora package.

In [12]:
from zklora import export_lora_submodules, generate_proofs, batch_verify_proofs

We will also use the Hugging Face Transformers and Peft libraries to load the model and tokenizer.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

We will use the `distilgpt2` model as the base model and the `q1e123/peft-starcoder-lora-a100` model as the LoRA model.

In [ ]:
base_model_name = "distilgpt2"
lora_model_name = "q1e123/peft-starcoder-lora-a100"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
lora_model = PeftModel.from_pretrained(base_model, lora_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
lora_model.eval()

# Activations from the Base Model

In order to prove the forward pass of the LoRA model, we need to export the activations from the base model.

To that end, we call the `export_lora_submodules` function. The `export_lora_submodules` function will export the activations from the base model for the given input texts and submodule key.

An input text is required for the base model to generate the activations. The activations are exported to the `intermediate_activations` directory as `json` files.

In the directory `lora_onnx_params` we will find the ONNX files for the LoRA model. The input text is also required to generate the ONNX files.

With the `submodule_key` we specify the name of the submodule we want to export. In this case, we want to export the activations from the `attn.c_attn` submodule.

In [14]:
texts = ["Hello from LoRA", "And another test", "One more line..."]

export_lora_submodules(
    model=lora_model,
    tokenizer=tokenizer,
    input_texts=texts,
    submodule_key="attn.c_attn",
)

After calling the `export_lora_submodules` function, we will see two directories created: `intermediate_activations` and `lora_onnx_params`.

The `intermediate_activations` directory will contain the activations from the base model for the given input texts.

The `lora_onnx_params` directory will contain the ONNX files for the LoRA model.

The `export_lora_submodules` function is asynchronous, so it will return immediately and the activations will be generated in the background.


# Generating Proofs

Now we can generate the proofs. For that, we call the `generate_proofs` function. This function will generate a proof for each pair of ONNX and JSON files in the `lora_onnx_params` and `intermediate_activations` directories.

The `generate_proofs` function is asynchronous, so it will return immediately and the proofs will be generated in the background. However, to make this work, and since we are using a notebook, you'll need to run the cell with the await statement in an async context.

Since the generation of proofs is a time-consuming process, it will take around 10 minutes depending on the hardware.

In [ ]:
async def main():
    return await generate_proofs(verbose=True)

await main()

The `generate_proofs` function will create a directory called `proof_artifacts` in the current working directory. This directory will contain the proof artifacts for each pair of ONNX and JSON files.

# Verifying Proofs

Now we can verify the proofs. For that, we call the `batch_verify_proofs` function. This function will verify the proofs generated that are stored in the `proof_artifacts` directory.

In [ ]:
batch_verify_proofs(verbose=True)